In [ ]:
!pip install xlrd

In [ ]:
!pip install lightgbm

In [ ]:
!pip install scipy==1.8.0

In [ ]:
!pip install deep-forest

In [ ]:
!pip install -U scikit-learn==1.5.0

In [ ]:
!pip install numpy==1.19.5

In [ ]:
!pip install -U pandas==1.3.0

In [ ]:
!pip install xgboost

In [ ]:
!pip install -U tensorflow

In [ ]:
!pip install scikit-learn

In [1]:
import pandas as pd
import numpy as np
from os.path import join

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn


def printf(*args, fname="log.txt"):
    with open(join("test_outputs",fname),"a+") as f:
        for a in args:
            f.write(str(a) + " ")
        f.write("\n") 
    print(args) 

# Load Reynolds

## load daily soil temperature

In [2]:
import os
fdir = "data/Reynolds/soiltemperature"

def loadReynolds(fname):
    df = pd.read_csv(fname, delim_whitespace=True,comment="#", encoding="ISO-8859-1",on_bad_lines="warn",header=None)
    df = df[df[3]==17]
    df_time = pd.to_datetime(df[0].astype(str) +  ' ' + df[1].astype(str) + ' ' + df[2].astype(str),format="%m %d %Y")
    
    df2 = df[[5,6,7,8,9,10,11,12]].replace('.', np.nan).astype(float).ffill(axis=0)

    df2[1] = fname.replace(fdir,"").replace("/hourly","").replace("soiltemperature.txt","")
    
    return pd.concat([df_time,df2], axis=1) 

soil_temp = []
for file in os.listdir(fdir):
    if file.endswith(".txt") and file.find("hourly") > -1:
        print(file)
        path = os.path.join(fdir, file)
        soil_temp += [loadReynolds(path)]

reynolds_soil_temp = pd.concat(soil_temp, axis=0)

hourly057x96soiltemperature.txt
hourly076x59soiltemperature.txt
hourly098x97soiltemperature.txt
hourly127x07soiltemperature.txt
hourly176x14soiltemperature.txt


In [3]:
fdir2 = "data/Reynolds"

def loadReynoldsCL(fname):
    df = pd.read_csv(fname, delim_whitespace=True,comment="#", encoding="ISO-8859-1",on_bad_lines="warn",header=None)
    df = df[(df[2] > 1984)|(df[2] == 1984) & (df[0] == 12) &(df[1] > 4)]
    df_time = pd.to_datetime(df[0].astype(str) +  ' ' + df[1].astype(str) + ' ' + df[2].astype(str),format="%m %d %Y")
    
    df2 = df[[3,4]].replace('.', np.nan).astype(float).ffill(axis=0)

    df2[1] = fname.replace(fdir2,"").replace("/daily","").replace("climate.txt","")
    
    return pd.concat([df_time,df2], axis=1) 

climate = []
for file in os.listdir(fdir2):
    if file.endswith(".txt") and file.find("daily") > -1:
        print(file)
        path = os.path.join(fdir2, file)
        climate += [loadReynoldsCL(path)]

reynolds_climate = pd.concat(climate, axis=0)

daily076x59climate.txt
daily127x07climate.txt
daily176x14climate.txt


In [4]:
reynolds_soil_temp = reynolds_soil_temp.rename(columns={0:'DATE',1:"LOC",5:"y_1",6:"y_2",7:"y_3",8:"y_4",9:"y_5",10:"y_6",11:"y_7",12:"y_8",3: 20, 4: 21}) 
reynolds_climate = reynolds_climate.rename(columns={0:'DATE',1:"LOC"})
climate = reynolds_climate.set_index(['DATE','LOC']).rename(columns={3: "C1", 4: "C2"}) 
soil = reynolds_soil_temp.set_index(['DATE','LOC'])

In [5]:
all_reynolds_data = pd.merge(soil,climate,left_index=True, right_index=True)

In [6]:
all_reynolds_data.to_csv("all_reynolds_data.csv",sep=";")

In [7]:
all_reynolds_data

,,y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,C1,C2
DATE,LOC,,,,,,,,,,
1985-08-02,076x59,NaN,28.2,24.9,NaN,21.9,22.0,22.2,NaN,9.4,22.6
1985-08-03,076x59,NaN,31.3,27.0,NaN,22.5,21.9,21.9,NaN,6.5,25.3
1985-08-04,076x59,NaN,30.8,27.2,NaN,23.2,22.3,22.1,NaN,8.6,32.2
1985-08-05,076x59,NaN,31.1,27.8,NaN,24.0,22.9,22.4,NaN,14.9,28.1
1985-08-06,076x59,NaN,33.1,29.0,NaN,24.3,23.1,22.6,NaN,11.0,31.0
...,...,...,...,...,...,...,...,...,...,...,...
1996-09-26,176x14,15.4,12.5,9.5,9.6,10.2,10.4,10.8,11.0,-1.1,9.6
1996-09-27,176x14,18.4,14.8,10.3,10.2,10.2,10.4,10.7,10.9,1.9,17.0
1996-09-28,176x14,19.9,16.6,12.0,11.7,11.0,11.0,10.9,10.9,10.1,21.2


# UK

In [8]:
uk_data = "data/UK/catalogue.ceh.ac.uk/datastore/eidchub/399ed9b1-bf59-4d85-9832-ee4d29f49bfb/"
climate_soil = []

def loadUK(fname):
    df = pd.read_csv(fname, sep=",",comment="#", on_bad_lines="warn")

    df1 = df.iloc[:, :2]
    df2 = df.iloc[:, 2:]    
    df2 = df2.astype(float).replace(-9999.0, np.nan).ffill(axis=0)    
    
    return pd.concat([df1,df2], axis=1)
    

for file in os.listdir(uk_data):
    if file.endswith(".csv") and file.find("daily") > -1 and file.find("flags") == -1 and file.find("metadata") == -1:
        print(file)
        path = os.path.join(uk_data, file)
        climate_soil += [loadUK(path)]

climate_soil =  pd.concat(climate_soil, axis=0)
climate_soil = climate_soil.rename(columns={"DATE_TIME":"DATE","SITE_ID":"LOC","TDT1_TSOIL":"y_1","TDT2_TSOIL":"y_2","TDT3_TSOIL":"y_3","TDT4_TSOIL":"y_4","TDT5_TSOIL":"y_5","TDT6_TSOIL":"y_6","TDT7_TSOIL":"y_7","TDT8_TSOIL":"y_8","TDT9_TSOIL":"y_9","TDT10_TSOIL":"y_10"}) 
climate_soil['DATE']= pd.to_datetime(climate_soil['DATE'])
climate_soil.set_index(["DATE","LOC"])

cosmos-uk_alic1_hydrosoil_daily_2015-2023.csv
cosmos-uk_wyth1_hydrosoil_daily_2013-2023.csv
cosmos-uk_sydlg_hydrosoil_daily_2018-2023.csv
cosmos-uk_gisbn_hydrosoil_daily_2014-2023.csv
cosmos-uk_riseh_hydrosoil_daily_2016-2023.csv
cosmos-uk_bickl_hydrosoil_daily_2015-2023.csv
cosmos-uk_morem_hydrosoil_daily_2018-2023.csv
cosmos-uk_portn_hydrosoil_daily_2014-2023.csv
cosmos-uk_stght_hydrosoil_daily_2015-2023.csv
cosmos-uk_glenw_hydrosoil_daily_2016-2023.csv
cosmos-uk_tadhm_hydrosoil_daily_2014-2023.csv
cosmos-uk_wimpl_hydrosoil_daily_2019-2023.csv
cosmos-uk_nwyke_hydrosoil_daily_2014-2023.csv
cosmos-uk_fivet_hydrosoil_daily_2018-2023.csv
cosmos-uk_hlacy_hydrosoil_daily_2018-2023.csv
cosmos-uk_plynl_hydrosoil_daily_2014-2023.csv
cosmos-uk_glens_hydrosoil_daily_2014-2023.csv
cosmos-uk_eastb_hydrosoil_daily_2014-2023.csv
cosmos-uk_holln_hydrosoil_daily_2014-2023.csv
cosmos-uk_finch_hydrosoil_daily_2017-2023.csv
cosmos-uk_sourh_hydrosoil_daily_2014-2023.csv
cosmos-uk_lodtn_hydrosoil_daily_20

,,LWIN,LWOUT,SWIN,SWOUT,RN,PRECIP,PRECIP_TIPPING,PRECIP_RAINE,PA,TA,...,STP_TSOIL50,COSMOS_VWC,CTS_MOD_CORR,D86_75M,SNOW,SNOW_DEPTH,SWE,ALBEDO,PE,GCC
DATE,LOC,,,,,,,,,,,,,,,,,,,,,
2015-03-06,ALIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-07,ALIC1,23.7,30.5,13.3,1.9,4.6,0.0,NaN,NaN,1012.2,8.6,...,5.6,41.2,1127.86170,22.12836,0.0,NaN,NaN,0.126,1.7,NaN
2015-03-08,ALIC1,27.1,30.5,4.8,0.7,0.7,0.0,NaN,NaN,1010.3,8.4,...,5.7,43.2,1118.28578,21.68494,0.0,NaN,NaN,0.116,0.6,NaN
2015-03-09,ALIC1,28.3,29.7,2.4,0.4,0.7,0.1,NaN,NaN,1011.8,6.2,...,5.9,39.1,1138.72068,22.63179,0.0,NaN,NaN,0.111,0.3,NaN
2015-03-10,ALIC1,23.5,29.8,12.4,1.8,4.3,0.0,NaN,NaN,1016.6,7.2,...,5.9,46.3,1105.15844,21.05842,0.0,NaN,NaN,0.121,1.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,CGARW,29.8,30.8,0.6,0.2,-0.6,38.7,NaN,NaN,964.9,8.8,...,9.1,66.2,1419.61285,16.04571,0.0,0.1,1.81413,0.153,0.3,NaN
2023-12-28,CGARW,28.6,30.5,1.0,0.3,-1.1,10.2,NaN,NaN,968.1,8.4,...,9.1,55.0,1462.74408,17.23641,0.0,0.1,1.81413,0.238,0.8,NaN
2023-12-29,CGARW,25.6,29.4,2.8,0.9,-1.9,3.9,NaN,NaN,973.1,6.3,...,9.0,50.3,1485.48209,17.86281,0.0,0.1,1.81413,0.274,0.8,NaN


In [9]:
climate_soil.rename(columns={"DATE_TIME":"DATE","SITE_ID":"LOC","TDT1_TSOIL":"y_1","TDT2_TSOIL":"y_2","TDT3_TSOIL":"y_3","TDT4_TSOIL":"y_4","TDT5_TSOIL":"y_5","TDT6_TSOIL":"y_6","TDT7_TSOIL":"y_7","TDT8_TSOIL":"y_8","TDT9_TSOIL":"y_9","TDT10_TSOIL":"y_10"}) 
climate_soil = climate_soil.set_index(["DATE","LOC"])


In [10]:
climate_soil.to_csv("uk_soil.csv",sep=";")

In [11]:
climate_soil

,,LWIN,LWOUT,SWIN,SWOUT,RN,PRECIP,PRECIP_TIPPING,PRECIP_RAINE,PA,TA,...,STP_TSOIL50,COSMOS_VWC,CTS_MOD_CORR,D86_75M,SNOW,SNOW_DEPTH,SWE,ALBEDO,PE,GCC
DATE,LOC,,,,,,,,,,,,,,,,,,,,,
2015-03-06,ALIC1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-07,ALIC1,23.7,30.5,13.3,1.9,4.6,0.0,NaN,NaN,1012.2,8.6,...,5.6,41.2,1127.86170,22.12836,0.0,NaN,NaN,0.126,1.7,NaN
2015-03-08,ALIC1,27.1,30.5,4.8,0.7,0.7,0.0,NaN,NaN,1010.3,8.4,...,5.7,43.2,1118.28578,21.68494,0.0,NaN,NaN,0.116,0.6,NaN
2015-03-09,ALIC1,28.3,29.7,2.4,0.4,0.7,0.1,NaN,NaN,1011.8,6.2,...,5.9,39.1,1138.72068,22.63179,0.0,NaN,NaN,0.111,0.3,NaN
2015-03-10,ALIC1,23.5,29.8,12.4,1.8,4.3,0.0,NaN,NaN,1016.6,7.2,...,5.9,46.3,1105.15844,21.05842,0.0,NaN,NaN,0.121,1.6,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-27,CGARW,29.8,30.8,0.6,0.2,-0.6,38.7,NaN,NaN,964.9,8.8,...,9.1,66.2,1419.61285,16.04571,0.0,0.1,1.81413,0.153,0.3,NaN
2023-12-28,CGARW,28.6,30.5,1.0,0.3,-1.1,10.2,NaN,NaN,968.1,8.4,...,9.1,55.0,1462.74408,17.23641,0.0,0.1,1.81413,0.238,0.8,NaN
2023-12-29,CGARW,25.6,29.4,2.8,0.9,-1.9,3.9,NaN,NaN,973.1,6.3,...,9.0,50.3,1485.48209,17.86281,0.0,0.1,1.81413,0.274,0.8,NaN


In [12]:
climate_soil = climate_soil.drop(["SNOW_DEPTH", "TDT1_VWC","TDT2_VWC","TDT3_VWC","TDT4_VWC","TDT5_VWC","TDT6_VWC","TDT7_VWC","TDT8_VWC","TDT9_VWC","TDT10_VWC","PRECIP_TIPPING","PRECIP_RAINE"],axis=1)

# Preprocessing of the Datasets

### Split onto 28 day long fragments. Remove fragments, which do not containt more than 30% of target values. 

### We do not consider temperatures for other soil layers as fatures deliberatly. 

In [ ]:
from datetime import date, timedelta
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

PERIOD = 28
TRHX = 0.3

def daterange(start_date, end_date, step):
    for n in range(0,int((end_date - start_date).days), step):
        yield start_date + timedelta(n)

all_data = []
for df in [all_reynolds_data, climate_soil]:
    locations = df.index.get_level_values(1)
    
    rdatasX = {}
    rdatasY = {}
    for l in set(locations):
        ld = df.query("LOC == '" + l + "'").reset_index().set_index("DATE").drop("LOC",axis=1)
        all_start = ld.index.min()
        all_end = ld.index.max()
    
        for start in daterange(all_start, all_end, PERIOD):
            end = start + timedelta(PERIOD)
            period_data = ld.loc[start:end]
            
            y_columns = set([c for c in df.columns if c.find("y_") > -1])
            not_y_columns = [c for c in df.columns if c.find("y_") == -1]
            for y_counter in y_columns:
                if y_counter not in rdatasX: 
                    rdatasX[y_counter] = []
                    rdatasY[y_counter] = []
                    
                y_ = period_data[y_counter].to_numpy().astype(float)
                y = y_[1:]
    
                nans = np.count_nonzero(np.isnan(y))
    
                if float(nans) / y.shape[0] < TRHX and y.shape[0] == PERIOD:
                    X = period_data[not_y_columns].to_numpy()[:-1].astype(float)
                    X = np.hstack([X,y_[:-1].reshape(-1,1)]) # add current y as a feature
                    old_dim = X.shape[1]
                    X = imp.fit_transform(X)
                    if X.shape[1] == old_dim:
                        nans = np.count_nonzero(np.isnan(X))
                        if nans == 0:
                            y = imp.fit_transform(y.reshape(-1,1)).reshape(y.shape)
                            rdatasX[y_counter].append(X)
                            rdatasY[y_counter].append(y)
                    
    all_data.append({k:[np.asarray(rdatasX[k]), np.asarray(rdatasY[k])] for k in rdatasX})

In [ ]:
all_data[0].keys()

In [ ]:
from sklearn.preprocessing import normalize

for k in all_data[0]:
    all_data[0][k][0] = normalize(all_data[0][k][0].reshape(-1,all_data[0][k][0].shape[2]),axis=0).reshape(all_data[0][k][0].shape)
    
for k in all_data[1]:
    all_data[1][k][0] = normalize(all_data[1][k][0].reshape(-1,all_data[1][k][0].shape[2]),axis=0).reshape(all_data[1][k][0].shape)

In [ ]:
from sklearn.model_selection import train_test_split
dict_data = {}
dict_data["Reynolds"] = {}
dict_data["UK"] = {}

for k in all_data[0]:
    x01,x02,y01,y02 = train_test_split(all_data[0][k][0], all_data[0][k][1], test_size=0.3,random_state=42)
    dict_data["Reynolds"][k] = {"train":{"X":x01,"y":y01},"test":{"X":x02,"y":y02}}
    
for k in all_data[1]:
    x11,x12,y11,y12 = train_test_split(all_data[1][k][0], all_data[1][k][1],test_size=0.3,random_state=42)
    dict_data["UK"][k] = {"train":{"X":x11,"y":y11},"test":{"X":x12,"y":y12}} 

all_data = dict_data


# Experiments

In [ ]:
!pip install xgboost

In [ ]:
!pip install protobuf==4.21.12

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install -U tkan

In [ ]:
!pip install optuna

In [ ]:
!pip install scikit-learn

## Neural Networks

In [ ]:
import tensorflow as tf
import optuna
from tkan import TKAN, BSplineActivation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from tensorflow.keras import regularizers
from sklearn.model_selection import KFold

nn_data = []


def make_modelLSTM(input_shape, hidden_size, dropout):
    input_layer = tf.keras.layers.Input(input_shape)
    #dim = tf.zeros([batch_size,hidden_size])  
    output_layer = tf.keras.layers.LSTM(hidden_size, return_sequences=True,dropout=dropout)(input_layer)#, initial_state=[dim, dim])
    output_layer2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))(output_layer)    
    return tf.keras.models.Model(inputs=input_layer, outputs=output_layer2)
 
def make_GRU(input_shape, hidden_size, dropout):
    input_layer = tf.keras.layers.Input(input_shape)
    output_layer = tf.keras.layers.GRU(hidden_size, return_sequences=True,dropout=dropout)(input_layer)#, initial_state=[dim, dim])
    output_layer2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear'))(output_layer)    
    return tf.keras.models.Model(inputs=input_layer, outputs=output_layer2)

def make_modelTKAN(input_shape, hidden_size, dropout):
    model = tf.keras.Sequential([
          tf.keras.layers.InputLayer(input_shape=input_shape),
          TKAN(hidden_size, tkan_activations=[BSplineActivation(3)], return_sequences=True, use_bias=True, kernel_regularizer=regularizers.L2(dropout),
    bias_regularizer=regularizers.L2(dropout), recurrent_regularizer=regularizers.L2(dropout)),
          tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, activation='linear')),
    ])
    return model

models = {"LSTM":make_modelLSTM, "GRU": make_GRU, "TKAN": make_modelTKAN}

for model_name in models:
    make_model = models[model_name]
    for ds_name in all_data:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"]
            Y_test = dat["test"]["y"]
            batch_size = 4       
    
            def objective(trial):
                lr = trial.suggest_float('lr', 0.0001, 0.01)
                hidden_size = trial.suggest_int('hs', 2, 8)
                en = trial.suggest_int('en', 10, 400)

                if model_name == "TKAN":
                    do = trial.suggest_float('dropout', 1e-5, 1e-2)
                else:    
                    do = trial.suggest_float('dropout', 0.05, 0.2)
                    
                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = make_model(input_shape=x_train.shape[1:],hidden_size=hidden_size, dropout = do)
                # 
                    opt = tf.keras.optimizers.Adam(learning_rate=lr)
                
                    model.compile(
                         optimizer=opt,
                         loss="mean_squared_error",
                         metrics=["mean_squared_error"],
                    )
                    
                    history = model.fit(
                         x_train[train_index],
                         Y_train[train_index],
                         batch_size=batch_size,
                         epochs=en,
                         verbose=0,
                    )
                    try:
                        y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                        scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                    except:
                        scores.append(500)
                return np.asarray(scores).mean() 
                
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=20)    
            
            lr = study.best_trial.params["lr"]     
            hs = study.best_trial.params["hs"]     
            en = study.best_trial.params["en"]   
            do = study.best_trial.params["dropout"]   
    
            model = make_model(input_shape=x_train.shape[1:],hidden_size=hs, dropout = do)
        # 
            opt = tf.keras.optimizers.Adam(learning_rate=lr)
        
            model.compile(
                 optimizer=opt,
                 loss="mean_squared_error",
                 metrics=["mean_squared_error"],
            )
            
            history = model.fit(
                 x_train,
                 Y_train,
                 batch_size=batch_size,
                 epochs=en,
                 verbose=0,
            )        
            try:
                y_pred = model.predict(x_test) #, batch_size=batch_size)
                mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
                mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
                printf(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max(),fname="networks_output.txt")     
                nn_data.append([model_name,ds_name,depth,mse_score, mae_score])
            except:
                pass
            del model



## Boosting

In [ ]:
!pip install deep-forest

In [ ]:
!pip install cython

In [ ]:
!pip install optuna

In [ ]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelXGB(max_depth,layers,C):
    return xgb.XGBRegressor(max_depth = max_depth, n_estimators = layers)

def make_modelCascade(max_depth,layers,C):
    return CascadeForestRegressor(max_depth = max_depth, max_layers = layers, n_estimators=4)

def make_modelBoosted(max_depth,layers,C):
    return CascadeBoostingRegressor(C=C, n_layers=layers, n_estimators = 1, max_depth=max_depth, n_iter_no_change = 1, validation_fraction = 0.1, learning_rate = 0.9)


models = {"XGB":make_modelXGB,"Cascade Forest":make_modelCascade, "Boosted Forest": make_modelBoosted}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in all_data:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2])
            x_test = dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2])
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                layers = trial.suggest_int('layers', 5, 15)
                max_depth = trial.suggest_int('max_depth', 1, 2)

                if model_name == "Boosted Forest":
                    C = trial.suggest_int('C', 1, 2000)
                else:
                    C = 0

                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = make_model(max_depth,layers,C)
                    
                    model.fit(
                         x_train[train_index],
                         Y_train[train_index],
                    )
                    y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                    scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)    
            
            layers = study.best_trial.params["layers"]  
            max_depth = study.best_trial.params["max_depth"]  

            if model_name == "Boosted Forest":
                C = study.best_trial.params["C"]  
            else:
                C = 0
            model = make_model(max_depth,layers,C)
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            printf(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max(),fname="boosting_output.txt")     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
    

# Adaptive weighing (AWDF and ECDFR)

In [ ]:
#ECDFR

In [ ]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import optuna

from ecdfr.gcForest import gcForest

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelECDFR(max_depth,layers,resampling_rate, et):
    config = {"estimator_configs":[{"n_fold": 5,"type":None,"max_depth":max_depth},{"n_fold": 5,"type":None,"max_depth":max_depth},{"n_fold": 5,"type":None,"max_depth":max_depth},{"n_fold": 5,"type":None,"max_depth":max_depth}],
              "error_threshold": et,
              "resampling_rate": resampling_rate,
              "random_state":None,
              "max_layers":layers,
              "early_stop_rounds":1,
              "train_evaluation":r2_score}
    
    return gcForest(config,2)

models = {"ecdfr":make_modelECDFR}

bo_data = []    
work_pair = []
best_pair = []

max_score = 100


for model_name in models:
    make_model = models[model_name]
    for ds_name in all_data:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2])
            x_test = dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2])
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                global max_score
                layers = trial.suggest_int('layers', 3, 15)
                max_depth = trial.suggest_int('max_depth', 1, 2)

                C = trial.suggest_float('resampling_rate', 0.1, 4)
                min_et = 0.5 * C - 1
                if min_et <= 0:
                    min_et = 0.05
                else:
                    if min_et > 0.99:
                        min_et = 0.99

                max_et = C -1.

                if max_et > 0.99:
                    max_et = 0.99
                else:
                    if max_et <=0:
                        max_et = 0.1

                if min_et >= max_et:
                    max_et = min_et + 0.001
                    
                et = trial.suggest_float('et', 0.05, 0.95)
                
                kf = KFold(n_splits=3)
                scores = []
                try:
                    for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                        model = make_modelECDFR(max_depth,layers,C,et)
                    
                        model.fit(
                             x_train[train_index],
                             Y_train[train_index],
                        )
                        y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                        scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                        sc = mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten())
                        if max_score == 100:
                            max_score = sc
                        else:
                            if sc > max_score:
                                max_score = sc
                                
                        work_pair.append([C,et])
                except Exception as e:
                    print(e)
                    scores = [max_score]
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)    
            
            layers = study.best_trial.params["layers"]  
            max_depth = study.best_trial.params["max_depth"]  


            C = study.best_trial.params["resampling_rate"]  
            et = study.best_trial.params["et"]  

            best_pair.append([C,et])
            model = make_model(max_depth,layers,C,et)
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            printf(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max(),fname="ecdfr_output.txt")     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
    

In [ ]:
from matplotlib import pyplot as plt

plt.scatter(*zip(*work_pair))

plt.show()

In [ ]:
# AWDF

In [ ]:
!python --version

In [ ]:
from matplotlib import pyplot as plt

plt.scatter(*zip(*best_pair))

plt.show()

In [ ]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelCascade(max_depth,layers,C,wt):
    wf = {0:"linear", 1:"1-w^1/2", 2:"1-w2"}
    return CascadeForestRegressor(max_depth = max_depth, max_layers = layers, n_estimators=4,adaptive=True,weighting_function = wf[wt],verbose=0,trx=1.0)


models = {"AWDF":make_modelCascade}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in all_data:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2])
            x_test = dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2])
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                layers = trial.suggest_int('layers', 5, 15)
                max_depth = trial.suggest_int('max_depth', 1, 2)
                wt = trial.suggest_int('weight_function', 0, 2)   
                if model_name == "Boosted Forest":
                    C = trial.suggest_int('C', 1, 2000)
                else:
                    C = 0

                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = make_model(max_depth,layers,C,wt)
                    
                    model.fit(
                         x_train[train_index],
                         Y_train[train_index],
                    )
                    y_pred = model.predict_sampled(x_train[test_index]) #, batch_size=batch_size)
                    scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)    
            
            layers = study.best_trial.params["layers"]  
            max_depth = study.best_trial.params["max_depth"]  
            wt = study.best_trial.params['weight_function']
            if model_name == "Boosted Forest":
                C = study.best_trial.params["C"]  
            else:
                C = 0
            model = make_model(max_depth,layers,C,wt)
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict_sampled(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            printf(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max(),fname="awdf_output.txt")     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
    

In [ ]:
!pip install -U numpy==1.22.4

In [ ]:
!pip install statsmodels

# ARIMA

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelARIMA(p,d,q,y):
    return SARIMAX(y.flatten(), order=(p,d,q),seasonal_order=(1,0,1,7),trend='c')


models = {"ARIMA": make_modelARIMA}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in ['UK']:#all_data:
        for depth in ["y_9"]:#all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"]
            Y_test = dat["test"]["y"]

            print(Y_train.shape,Y_test.shape)

            def objective(trial):
                p = trial.suggest_int('p', 1, 5)
                d = trial.suggest_int('d', 0, 5)
                q = trial.suggest_int('q', 0, 5)

                kf = KFold(n_splits=3)
                scores = []
                try:
                    for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                        for y_t in Y_train[train_index]:
                            model = make_model(p,d,q, y_t[:27].flatten())
                            res = model.fit() 
                            y_pred = res.forecast(steps = 1) #, batch_size=batch_size)
                            
                            scores.append(mean_squared_error(np.asarray([y_t[27]]),y_pred))
                except:
                    scores.append(1000)

                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=100)    

            try:
                p = study.best_trial.params["p"]  
                d = study.best_trial.params["d"]  
                q = study.best_trial.params["q"]  

                mse_list = []
                mae_list = []
                
                for y_t in Y_test:
                    model = make_model(p,d,q,y_t[:27])
                    res = model.fit() 
                    y_pred = res.forecast(steps = 1) #, batch_size=batch_size)
                    mse_list.append(mean_squared_error(np.asarray([y_t[27]]),y_pred))
                    mae_list.append(mean_absolute_error(np.asarray([y_t[27]]),y_pred))
                    
                mse_score = np.asarray(mse_list).mean()
                mae_score = np.asarray(mae_list).mean()
                printf(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max(),"ARIMA_output.txt")     
                bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
            except:
                pass
    

#print (model.mle_retvals)

[I 2025-01-23 11:18:43,918] A new study created in memory with name: no-name-4590ad57-b536-44a8-ac30-996b61c2a8d3


(891, 28) (382, 28)
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.94249D+00    |proj g|=  3.19791D+01


 This problem is unconstrained.



At iterate    5    f=  4.50059D-02    |proj g|=  3.38169D-01

At iterate   10    f= -5.34628D-02    |proj g|=  7.24377D-02

At iterate   15    f= -6.81303D-02    |proj g|=  1.75061D-01

At iterate   20    f= -9.52430D-02    |proj g|=  1.05228D-01

At iterate   25    f= -9.85188D-02    |proj g|=  5.74928D-02

At iterate   30    f= -1.17493D-01    |proj g|=  1.64771D-02

At iterate   35    f= -1.18161D-01    |proj g|=  1.21401D-02

At iterate   40    f= -1.18219D-01    |proj g|=  1.45481D-02

At iterate   45    f= -1.18276D-01    |proj g|=  1.59695D-02

At iterate   50    f= -1.18346D-01    |proj g|=  5.44098D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

 This problem is unconstrained.


...


At iterate   50    f= -8.27848D-01    |proj g|=  3.88155D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     50     67      1     0     0   3.882D-01  -8.278D-01
  F = -0.82784753163114089     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.92439D+02    |proj g|=  1.84617D+03

At iterate    5    f=  2.14778D+00    |proj g|=  6.94971D-01

At iterate   10    f=  8.23823D-01    |proj g|=  7.62998D-01

At iter

 This problem is unconstrained.



At iterate   45    f=  2.39499D-03    |proj g|=  3.17549D-03

At iterate   50    f=  2.25720D-03    |proj g|=  4.38633D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     50     67      1     0     0   4.386D-02   2.257D-03
  F =   2.2571973077992933E-003

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f= -8.60624D-01    |proj g|=  2.63501D+00

At iterate    5    f= -8.79132D-01    |proj g|=  4.69703D-01

At iter

 This problem is unconstrained.



At iterate   45    f= -9.27837D-01    |proj g|=  2.18210D-01

At iterate   50    f= -9.28076D-01    |proj g|=  2.00667D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     50     63      1     0     0   2.007D-02  -9.281D-01
  F = -0.92807618242065559     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.36577D-02    |proj g|=  4.93558D+00

At iterate    5    f= -1.13634D-01    |proj g|=  2.59932D-01

At iter

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
rf_data = [] 

for ds_name in all_data:
    for depth in all_data[ds_name]:
        best = 1000
        best_d = 2
        for n_estimators in [100]:
            for tdepth in [2,3,5,7,10,12]:
                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = RandomForestRegressor(n_estimators = n_estimators, max_depth = tdepth)
                    dat = all_data[ds_name][depth]
                    model.fit(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2])[train_index], dat["train"]["y"].flatten()[train_index])
                    y_pred = model.predict(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2]))[test_index])
                    scores.append(mean_squared_error(y_pred, dat["train"]["y"].flatten())[test_index])
                    
                if np.asarray(scores).mean() < best:
                    best = np.asarray(scores).mean()
                    best_d = tdepth

        model = RandomForestRegressor(n_estimators = n_estimators, max_depth = best_d)
        dat = all_data[ds_name][depth]
        model.fit(dat["train"]["X"].reshape(-1,dat["train"]["X"].shape[2]), dat["train"]["y"].flatten())
        y_pred = model.predict(dat["test"]["X"].reshape(-1,dat["test"]["X"].shape[2]))
        mse_score = mean_squared_error(y_pred, dat["test"]["y"].flatten())
        mae_score = mean_absolute_error(y_pred, dat["test"]["y"].flatten())        
                
        print("Random Forest", ds_name,depth,mse_score, mae_score, dat["test"]["y"].min(),dat["test"]["y"].max())
        rf_data.append(["Random Forest", ds_name,depth,mse_score, mae_score])